In [215]:

import pandas as pd
import json
from datetime import datetime, timedelta
df = pd.read_csv('data.csv')


In [216]:
import pandas as pd
import json
from datetime import datetime, timedelta

def calculate_claims(json_data):
    # Check if the contracts data is empty or NaN
    if pd.isna(json_data) or json_data.strip() == "":
        return -3
    
    try:
        # pass
        # Attempt to load the JSON data
        claims = json.loads(json_data)
        
        
        
        today = datetime.today()
        last_180_days = today - timedelta(days=780)
        
        if isinstance(claims, list):
            # print(type(claims))
       
            valid_claims = []
            for claim in claims:
                claim_date = claim.get('claim_date')
                
                if claim_date and claim_date.strip() != "":
                    try:
                        claim_date_parsed = datetime.strptime(claim_date, '%d.%m.%Y')
                        
                        if claim_date_parsed >= last_180_days:
                            valid_claims.append(claim)
                    except ValueError:
                        continue
            
            # Return the count of valid claims or -3 if none are found
            return len(valid_claims) if valid_claims else -3
        # print(type(claims))
    except (json.JSONDecodeError, TypeError):
        # Handle cases where JSON is malformed or empty
        return -3

# Apply the function to the contracts column
df['tot_claim_cnt_l180d']=df['contracts'].apply(calculate_claims)

In [218]:
def calculate_exposure(json_data):
    # Check if the contracts data is empty or NaN
    if pd.isna(json_data) or json_data.strip() == "":
        return -3
    
    try:
        # Attempt to load the JSON data
        loans = json.loads(json_data)
        
        # Filter valid loans
        valid_loans = []
        if isinstance(loans, list):
            for loan in loans:
                bank = loan.get('bank')
                loan_summa = loan.get('loan_summa', 0)
                contract_date = loan.get('contract_date')
                
                # Apply filtering conditions
                if bank not in ['LIZ', 'LOM', 'MKO', 'SUG', None] and contract_date:
                    valid_loans.append(loan_summa)
            
            # Check if there are no valid loans
            if not valid_loans:
                return -3
            
            # Check if all valid loans have a loan_summa of zero
            if all(summa == 0 for summa in valid_loans):
                return -1
            
            # Sum the loan_summa for all valid loans
            return sum(valid_loans)
    
    except (json.JSONDecodeError, TypeError):
        # Handle cases where JSON is malformed or empty
        return -3

# Apply the function to the contracts column
df['disb_bank_loan_wo_tbc'] = df['contracts'].apply(calculate_exposure)

In [ ]:
def day_sinlastloan(json_data):
    # Check if the contracts data is empty or NaN
    if pd.isna(json_data) or json_data.strip() == "":
        return -3
    
    try:
        # Attempt to load the JSON data
        loans = json.loads(json_data)
        
        # Filter valid loans
        valid_loans = []
        if isinstance(loans, list):
            for loan in loans:
                bank = loan.get('bank')
                loan_summa = loan.get('loan_summa', 0)
                contract_date = loan.get('contract_date')
                
                # Apply filtering conditions
                if bank not in ['LIZ', 'LOM', 'MKO', 'SUG', None] and contract_date:
                    valid_loans.append(loan_summa)
            
            # Check if there are no valid loans
            if not valid_loans:
                return -3
            
            # Check if all valid loans have a loan_summa of zero
            if all(summa == 0 for summa in valid_loans):
                return -1
            
            # Sum the loan_summa for all valid loans
            return sum(valid_loans)
    
    except (json.JSONDecodeError, TypeError):
        # Handle cases where JSON is malformed or empty
        return -3

# Apply the function to the contracts column
df['disb_bank_loan_wo_tbc'] = df['contracts'].apply(calculate_exposure)

In [219]:
def parse_date(date_str):
    date_formats = [
        '%Y-%m-%d %H:%M:%S.%f%z',  
        '%Y-%m-%d %H:%M:%S%z',   
        '%Y-%m-%d %H:%M:%S.%f',   
        '%Y-%m-%d %H:%M:%S',       
    ]
    
    for date_format in date_formats:
        try:
            return datetime.strptime(date_str, date_format)
        except ValueError:
            continue
    return None

def calculate_day_sinlastloan(contracts_json, application_date_str):
    if pd.isna(contracts_json) or contracts_json.strip() == "":
        return -3

    try:
        contracts = json.loads(contracts_json)

    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}, contracts_json: {contracts_json}")
        return -3  # Return -3 if JSON decoding fails

    application_date = parse_date(application_date_str)
    if application_date is None:
        print(f"Date parsing error: application_date_str: {application_date_str}")
        return -3  # Return -3 if date parsing fails

    # Check if contracts is a list of dictionaries or a single dictionary
    if isinstance(contracts, dict):
        contracts = [contracts]

    # Filter valid loans with non-null and non-empty summa and non-null contract_date
    valid_loans = [
        contract for contract in contracts
        if contract.get('summa') not in (None, '', '0') and contract.get('contract_date') not in (None, '')
    ]
    if all(loan['loan_summa'] == 0 for loan in valid_loans):
        return -1


    # Find the last loan based on the contract_date
    last_loan = None
    for contract in valid_loans:
        for date_format in ['%Y-%m-%d', '%d.%m.%Y']:
            try:
                contract_date = datetime.strptime(contract['contract_date'], date_format)
                if last_loan is None or contract_date > last_loan:
                    last_loan = contract_date
                break
            except ValueError:
                continue

    if last_loan is None:
        return -3  # No valid contract_date found

    # Convert last_loan to offset-aware datetime
    last_loan = last_loan.replace(tzinfo=application_date.tzinfo)

    # Calculate days since last loan
    days_since_last_loan = (application_date - last_loan).days

    return days_since_last_loan if days_since_last_loan >= 0 else -3

# Apply the function to each row in the DataFrame
df['day_sinlastloan'] = df.apply(
    lambda row: calculate_day_sinlastloan(row['contracts'], row['application_date']), axis=1
)


In [230]:
df1=df[['id','application_date','tot_claim_cnt_l180d','disb_bank_loan_wo_tbc','day_sinlastloan']]

In [232]:
df1.to_csv('contract_features.csv',index=False)